<a href="https://colab.research.google.com/github/smduarte/LiveFeeds/blob/master/assignment-1/assignment-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RC 2020/2021 - Assignment 1

## Goals
With this assignment students will get a better understanding of how packet switching networks work, what is the transit time of packets in this type of networks and the way to compute it, and how packet switchning network properties impact the end-to-end performance of sending information from a node to another one.

## Assumptions
In what follows we will consider, by hypothesis, that all links are perfect and never corrupt or loose packets. The same property applies to nodes - they are bullet proof and never crash or loose packets. Also, as we are using CNSS, all computations are performed instantly, without any delay.

Network configurations of the different experiments made in this assignment use links with the same characteristics: bandwidth of 2 M bits per second, or 2,000,000 bps, and a propagation time, or latency, of 20 ms, since they have 4,000 Km each (4000 Km / 200,000 Km per second = 4 x 10^3 / 2 x 10^5 Km per second = 2 x 10^-2 s = 20 ms).

## Understanding store & forward, end-to-end transit time and the time required to transfer information in a packet switched network

**WARNING 1: please study section 3.2 and 3.3 of the book supporting the course to fully understand this section.**

**WARNING 2: please do not forget to update CNSS to its lastest version.**

In what follows, we are going to make several experiments with the goal of understanding which factors contribute to the transit time of a packet in a network, as well as to get a first glimpse of what impacts the time required to transfer information from one node to a different one in a packet switched network.

The three network configurations used in the first set of experiments [configs/config1.1.txt](configs/config1.1.txt), [configs/config1.2.txt](configs/config1.2.txt) and 
[configs/config1.3.txt](configs/config1.3.txt) are depicted below:



![](https://github.com/jlegatheaux/RC2020-assignments/blob/master/assignment-1/figs/config1.1-3.png?raw=1)

In [33]:
%%writefile config1.1.txt
# A network with a sender node and a receiver node interconnected
# by a direct link. The link has 1 Mbps bandwidth and 10 ms latency


# uncomment if you want to see control algorithms traces
# parameter trace 


Node 0 1 cnss.lib.EndSystemControl FilesSender 10
Node 1 1 cnss.lib.EndSystemControl FilesReceiver

Link 0.0 1.0 2000000 20 0.0 0.0

Overwriting config1.1.txt


In [34]:
%%writefile config1.2.txt
# A network with a sender node and a receiver node interconnected
# by a switch. Both links have 1 Mbps bandwidth and 10 ms latency


# uncomment if you want to see control algorithms traces
# parameter trace   

Node 0 1 cnss.lib.EndSystemControl FilesSender 10
Node 1 1 cnss.lib.EndSystemControl FilesReceiver
Node 2 2 cnss.lib.FloodingSwitch cnss.lib.EmptyApp

Link 0.0 2.0 2000000 20 0.0 0.0
Link 1.0 2.1 2000000 20 0.0 0.0

Overwriting config1.2.txt


In [12]:
%%writefile config1.3.txt
# A network with a sender node and a receiver node interconnected
# by two switches. All links have 1 Mbps bandwidth and 10 ms latency

# uncomment if you want to see control algorithms traces
# parameter trace   

Node 0 1 cnss.lib.EndSystemControl FilesSender 10
Node 1 1 cnss.lib.EndSystemControl FilesReceiver
Node 2 2 cnss.lib.FloodingSwitch cnss.lib.EmptyApp
Node 3 2 cnss.lib.FloodingSwitch cnss.lib.EmptyApp

Link 0.0 2.0 2000000 20 0.0 0.0
Link 2.1 3.0 2000000 20 0.0 0.0
Link 3.1 1.0 2000000 20 0.0 0.0

Writing config1.3.txt


In [27]:
%%writefile FilesSender.java

import cnss.simulator.*;
import cnss.lib.*;

public class FilesSender extends AbstractApplicationAlgorithm {

	public static int BLOCKSIZE = 10000; // 10000*8 = 80000 bits
	public static int TOTAL_PACKETSIZE = BLOCKSIZE+Packet.HEADERSIZE;


	int totSent = 0;
	int filesize = 0;
	int totalBlocks = 0;

	public FilesSender() {
		super(true, "Files-sender");
	}

	public int initialise(int now, int node_id, Node self, String[] args) {
		super.initialise(now, node_id, self, args);
		log(0, "starting");
		if ( args.length != 1 ) {
			log(now, "ERROR: missing argument totalBlocks");
			System.exit(-1);
		}
		totalBlocks = Integer.parseInt(args[0]);
		for ( int i = 1; i <= totalBlocks; i++ ) {
			self.send( self.createDataPacket( 1, new byte[BLOCKSIZE]));
			log(now, "sent packet of size "+TOTAL_PACKETSIZE+" n. "+i);
		}
		self.set_timeout(60000); // 60 seconds later
		return 0;	
	}

	public void on_timeout(int now) {
		self.send( self.createDataPacket( 1, new byte[TOTAL_PACKETSIZE*totalBlocks]));
		log(now, "sent packet of size: "+TOTAL_PACKETSIZE*totalBlocks+" n. "+(totalBlocks+1));
	}

	public void on_receive(int now, DataPacket p) {
		log(now, "received ack "+p+ " w/ payload "+new String(p.getPayload()));
	}

	public void showState(int now) {
		System.out.println(name + " sent " + totSent + " packets with blocks");
	}
} 

Overwriting FilesSender.java


The sender code deserves only a few comments. 

Upon initialization, the file is transfered in multiple packets, as given
by the node's argument.

To send the file in just one big packet, a timeout is setup during initialisation to set an alarm for 60000 ms or 60 seconds later. The big packet is then sent when upcall on_timeout() is executed. 

In order to make both tranfer solutions comparable, when one only packet is sent, its size has been incresed n times the size of the header of a packet (n times 20 bytes). Therefore, the total number of bytes transfered with the two solutions only differs by 20 bytes (the size of the header of the big packet). 

The application code of the sender node is prepared to receive an ack from the receiver. However, with these three experiments, no acks are sent by node 1, the receiver. 

The application uses logging to show how the transfer is progressing. The application code of the receiver node also uses logging to show when packets are received, see below.

The receiver application is shown below. The most relevant part is its on_receive() upcall, which merely logs packet reception and the total number of packets already received.



In [30]:
%%writefile FilesReceiver.java

import cnss.simulator.*;
import cnss.lib.*;

public class FilesReceiver extends AbstractApplicationAlgorithm {
	
	int totReceived = 0;

  public FilesReceiver() {
      super(true, "Files-receiver");
  }

  public int initialise(int now, int node_id, Node self, String[] args) {
	  super.initialise(now, node_id, self, args);
	  log(0, "starting");
	  return 0;
	}

  public void on_receive( int now, DataPacket p ) {
	  totReceived++;
	  log(now, "got: " + p + " n. "+totReceived);
  }
  
	public void showState(int now) {
		System.out.println(name + " received "+totReceived+" packets with blocks");
	}
} 

Overwriting FilesReceiver.java


### First Experiment

In [36]:
%%bash

# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java


javac -cp .:cnss-classes *.java
java -cp .:cnss-classes cnss.simulator.Simulator config1.1.txt

Already up to date.
Loading configuration : config1.1.txt
Reading file config1.1.txt
Created Node 0: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: FilesSender
Created Node 1: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: FilesReceiver
Added link to node 0 - Link (Node1:0 I1:0)<-->(Node2:1 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 1 - Link (Node1:0 I1:0)<-->(Node2:1 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up

simulation starts - first processing step with clock = 0

log: Files-sender time 0 node 0 starting
log: Files-sender time 0 node 0 sent packet of size 10020 n. 1
log: Files-sender time 0 node 0 sent packet of size 10020 n. 2
log: Files-sender time 0 node 0 sent packet of size 10020 n. 3
log: Files-sender time 0 node 0 sent packet of size 10020 n. 4
log: Files-sender time 0 node 0 sent packet of size 10020 n. 5
log: Files-sender time 0 node 0 sent packet of size 10020 n. 6
log: Files-sender time 0 node 0

### Analysis

By following the output of the simulation, it is easy to observe that the 10th packet has been received at time 420. Thus, the file took 420 ms seconds to be transfered with the solution that sent 10 times 10,000 bytes packets. The same time it takes when the file is transferred in one only big packet of 100,000 bytes plus 200 bytes of headers.

You can easily compute analytically these results. 

In order to fully understand how that should be done, you must study sections 3.2 and 3.3 of Chapter 3 of the book of the course. 

Transmission time (Tt) of a packet with 10,000 Bytes (80,000 bits) to a link with 2 Mbps bit rate is 40 ms (Tt = size in bits / bit rate = 80000 / 2000000 = 0,040). The transmission time of the big packet is 10 times this and the propagation time of the link is 20 ms.


#### Second Experiment

After understanding everything you can now proceed to the next experiment, and give the following command:



In [35]:
%%bash
java -cp .:cnss-classes cnss.simulator.Simulator config1.2.txt

Loading configuration : config1.2.txt
Reading file config1.2.txt
Created Node 0: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: FilesSender
Created Node 1: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: FilesReceiver
Created Node 2: 2 interf.s, ctr code: cnss.lib.FloodingSwitch app code: cnss.lib.EmptyApp
Added link to node 0 - Link (Node1:0 I1:0)<-->(Node2:2 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 2 - Link (Node1:0 I1:0)<-->(Node2:2 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 1 - Link (Node1:1 I1:0)<-->(Node2:2 I2:1) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 2 - Link (Node1:1 I1:0)<-->(Node2:2 I2:1) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up

simulation starts - first processing step with clock = 0

log: Files-sender time 0 node 0 starting
log: Files-sender time 0 node 0 sent packet of size 10020 n. 1
log: Files-sender time 0 node 0 sent packet 

### Third Experiment

In [37]:
%%bash
java -cp .:cnss-classes cnss.simulator.Simulator config1.3.txt

Loading configuration : config1.3.txt
Reading file config1.3.txt
Created Node 0: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: FilesSender
Created Node 1: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: FilesReceiver
Created Node 2: 2 interf.s, ctr code: cnss.lib.FloodingSwitch app code: cnss.lib.EmptyApp
Created Node 3: 2 interf.s, ctr code: cnss.lib.FloodingSwitch app code: cnss.lib.EmptyApp
Added link to node 0 - Link (Node1:0 I1:0)<-->(Node2:2 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 2 - Link (Node1:0 I1:0)<-->(Node2:2 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 2 - Link (Node1:2 I1:1)<-->(Node2:3 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 3 - Link (Node1:2 I1:1)<-->(Node2:3 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
Added link to node 3 - Link (Node1:3 I1:1)<-->(Node2:1 I2:0) bwd: 2000000 bps lat: 20 ms error %: 0.0 jit %: 0.0 up
A

Now things become more interesting, since the times taken by the two transfers took are different. 

It is worth noting that in the Internet, as well as in almost all networks, it is not possible to send packets as big as the big packet used to send the file in one only packet. NCSS makes no restrictions on the size of packets because it is a simulation tool. Nevertheless, the experiments also show that it is not very interesting to use huge sized packets (this is a relative concept related to the bandwidth of links) as we will see next.

In fact, in experiment 2, when the file is sent in 10 packets, it takes 480 ms to get to the destination, instead of 420 in the first experiment, while it takes 540 ms in the third experiment. 

You should repeat the analytical analysis required to understand why these results are obtained. From experiment 1 to 2, the transfer time increased 60 ms, while in experiment 3 it increased 60 ms over experiment 2 and 120 ms over experiment 1.

The increase from one experiment to the following one is related to the extra transmission time introduced by the extra switch and link, added to the latency of the extra link. You can use the figure below to better understand the reasons that explain it.

![](figs/storeForward1.png)

As it is explained in the book, packet switched networks employ switches that use the store & forward principle, which states that packets must be fully received by a switch before being forwarded to the next one (or to an end-system). While the switch can send and receive several packets in parallel over different links, each packet can only be forwarded after being fully received, analysed and processed. It is only then that the outgoing interface in its way to the destination is choosen and its transmission may proceed. As such, if we replace one link by several links interconnected by switches, even if the sum of the latencies of the new links is equal to the latency of the replaced one, each link introduces an extra transmission time to the packet end-to-end transfer time.

If we look now at the results of the three experiments in what concerns the transfer of the file in one only big packet, more lessons can be learned. 

In experiment 1, the transfer using one only packet took 420 ms to complete, the same when several packets were used. However in experiment 2 the same transfer takes 840 ms and 1260 ms in experiment 3. From one experiment to the following, the transfer time increased 420 ms. That increase is also due to the same reason, a transmission time plus the latency of the extra switch and link introduced eah time. However, now, the transmission time of the big packet takes 400 ms instead of the 40 ms that each "small" packet took. 

The figure below illustrates quite clearly the difference between experiments 1 and 2. 

![](figs/storeForward2.png)

The lesson is, if links have bit rates that introduce significant transmission times, increasing the size of packets may introduce unexpected increases in transit time.

Before proceeding you should review the 3 experiments and take a sheet of paper and redo the calculations in the three cases to compare your computations with the results shown by the simulations. You should be convinced at the end, that computing transfer times in a network where no packets are lost, and there is no competing traffic (other sources sending packets that cross the same links that your packets also cross) is not difficult at all.

#### Further experiments

You can also repeat the same experiments with bigger files (more packets) or with links with higher bit rates. If you increase the bandwithd of links from 2 Mbps, to, for example, 100 Mbps or to 1000 Mbps (1 Gbps), the transmission times become very small. For example, sending 80,000 bits at 1 Mbps requires 80 ms, while sending the same packet at 1 Gbps only requires 0,08 ms or 80 micro seconds. As the bandwidth increases, the dominant factor in end-to-end transit time is links latency.

**Warning:** when changing a configuration file, you should pay attention to the fact that you may not make mistakes or otherwise NCSS crashes miserably. In particular, each token in the file must be separated from the next one by exactly one space character and you must not enter lines with only spaces. In a next version we will improve the configuration file parsing habilities of CNSS.

A last interesting observation concerning CNSS is related to the fact that these three configuration files have no stop parameter. Sometimes it is not required to introduce one, since CNSS recognizes that no more events can be fired in the simulation and stops its processing.